In [68]:
! python -V
!ls .
import datetime
print(datetime.datetime.now())
import time
print(time.time())

Python 2.7.17 :: Anaconda, Inc.
Programs API.ipynb      upload_from_csv.py      vericast_reports.py
Untitled.ipynb          upload_from_csv.pyc
__init__.py             uploading_epg_csv.ipynb
2020-02-27 19:09:37.575930
1582830577.58


The time value a:
- returned by:
    - `gmtime()`
    - `localtime()`
    - `strptime()`
- accepted by:
    - `asctime()`
    - `mktime()`
    - `strftime()`
    
is a sequence of 9 integers.

Return values of:

    - `gmtime()`
    - `localtime()`
    - `strptime()`
    
also offer attribute names for individual fields.

# Goals:
- Localize & convert to UTC engineers time.
```python
start_date='2018-01-29', 
start_time='12:00', 
end_date='2018-02-01',
end_time='00:00:00',
tz_offset='Europe/Madrid'

# Unaware
user_start_time='%sT%s00.000000'.format(start_date,start_time)
```


- And I want start_time_utc in epoch too

<hr>

## Engineer input

In [95]:
# from matches-%channel%.json
# local & unaware
start_time_local = datetime.datetime.strftime(user_start_time,'%Y-%m-%dT%H:%M:%f.000000')
# localize by adding timezone
print(start_time_local)

TypeError: descriptor 'strftime' requires a 'datetime.date' object but received a 'str'

In [93]:
# Create the DatetimeIndex 
# Here 'Q' represents quarter end frequency  
didx = pd.DatetimeIndex(report.df['localized_time']).tz_localize(tz ='Europe/Madrid')
didx
didx2 = pd.DatetimeIndex(report.df['converted_time']).tz_convert(tz ='Europe/Madrid')
didx2

NameError: name 'report' is not defined

## File input

In [20]:
line = {"start_time_utc": "2018-02-15T22:51:59.000000"}

In [80]:
# from matches-%channel%.json
start_time_utc = datetime.datetime.strptime(line['start_time_utc'],'%Y-%m-%dT%H:%M:%f.000000')
print(start_time_utc.replace(tz=datetime.timezone.utc))
# its UTC so we have to tell datetime it is UTC


AttributeError: 'module' object has no attribute 'timezone'

In [24]:
for item in line.iteritems():
    print(item)

('start_time_utc', '2018-02-15T22:51:59.000000')


In [29]:
start_time_utc_str = '2018-02-15T22:51:59.000000'
# parse this
start_time_utc = time.strftime('%Y-%M-%dT%H:%M:%S%s',)

In [30]:
time.gmtime()

'2020-52-27T18:52:111582829531'

In [33]:
import datetime
print(datetime.datetime.now())
print()

2020-02-27 18:57:23.835369
time.struct_time(tm_year=2020, tm_mon=2, tm_mday=27, tm_hour=18, tm_min=57, tm_sec=23, tm_wday=3, tm_yday=58, tm_isdst=0)
